<a href="https://colab.research.google.com/github/shweta-aiml/CourseraTensorFlow/blob/master/Random_Numbers_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NameError: ignored

In [52]:
!pip install tf-nightly-2.0-preview


     |████████████████████████████████| 95.2MB 34kB/s 
     |████████████████████████████████| 450kB 11.3MB/s 
     |████████████████████████████████| 3.8MB 43.5MB/s 
     |████████████████████████████████| 81kB 6.7MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
ERROR: tb-nightly 2.1.0a20191127 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2


In [2]:
import pandas as pd  
from random import random
import tensorflow as tf
tf.__version__

'2.0.0-dev20191002'

In [0]:
flow = (list(range(1,10,1)) + list(range(10,1,-1)))*1000  
pdata = pd.DataFrame({"a":flow, "b":flow})  
pdata.b = pdata.b.shift(9)  
data = pdata.iloc[10:] * random() 

In [4]:
data.size

35980

In [0]:
import numpy as np

def _load_data(data, n_prev = 100):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(df, test_size=0.1):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))

    X_train, y_train = _load_data(df.iloc[0:ntrn])
    X_test, y_test = _load_data(df.iloc[ntrn:])

    return (X_train, y_train), (X_test, y_test)

In [6]:
(X_train, y_train), (X_test, y_test) = train_test_split(data)  # retrieve data


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [7]:
X_train.shape,y_train.shape

((16091, 100, 2), (16091, 2))

In [0]:
X_train[0],y_train[0]

In [14]:

in_out_neurons = 2  
hidden_neurons = 300

model = tf.keras.models.Sequential()  
model.add(tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]))
model.add(tf.keras.layers.LSTM(in_out_neurons, return_sequences=False))  
model.add(tf.keras.layers.Dense(hidden_neurons,activation='linear'))  
model.compile(loss="mean_squared_error", optimizer="rmsprop") 
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, None, 1)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 2)                 32        
_________________________________________________________________
dense_2 (Dense)              (None, 300)               900       
Total params: 932
Trainable params: 932
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_train, y_train, batch_size=450, nb_epoch=10, validation_split=0.05)  


ValueError: ignored

In [0]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
  dataset = tf.data.Dataset.from_tensor_slices(series)
  dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
  dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
  dataset = dataset.batch(batch_size).prefetch(1)
  return dataset

In [0]:

window_size = 20
batch_size = 32
shuffle_buffer_size = 1000

In [0]:
tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)

tf.keras.backend.clear_session()
dataset = windowed_dataset(X_train, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(32,input_shape=(100,2), return_sequences=True),
  tf.keras.layers.LSTM(32),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100.0)
])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(X_train,y_train, epochs=100, callbacks=[lr_schedule])

Train on 16091 samples
Epoch 1/100
16091/16091 [==============================] - 40s 2ms/sample - loss: 0.9554 - mae: 1.4044
Epoch 2/100
16091/16091 [==============================] - 37s 2ms/sample - loss: 0.8717 - mae: 1.3215
Epoch 3/100
16091/16091 [==============================] - 38s 2ms/sample - loss: 0.7923 - mae: 1.2442
Epoch 4/100
16091/16091 [==============================] - 38s 2ms/sample - loss: 0.7185 - mae: 1.1659
Epoch 5/100
16091/16091 [==============================] - 38s 2ms/sample - loss: 0.6533 - mae: 1.0880
Epoch 6/100
16091/16091 [==============================] - 38s 2ms/sample - loss: 0.6012 - mae: 1.0215
Epoch 7/100
16091/16091 [==============================] - 38s 2ms/sample - loss: 0.5639 - mae: 0.9734
Epoch 8/100
16091/16091 [==============================] - 38s 2ms/sample - loss: 0.5404 - mae: 0.9432
Epoch 9/100
16091/16091 [==============================] - 38s 2ms/sample - loss: 0.5268 - mae: 0.9237
Epoch 10/100
16091/16091 [========================